In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import glob
from PIL import Image
from imgaug import augmenters as iaa

from keras.preprocessing.image import ImageDataGenerator

print("Tensorflow version " + tf.__version__)

In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 64

In [ ]:
# img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.resnet50.preprocess_input, input_shape=[IMAGE_SIZE, IMAGE_SIZE, 3])
    
# base_model = tf.keras.applications.ResNet50(weights=None, include_top=False)

    
# model = tf.keras.Sequential([

#         base_model,
#         tf.keras.layers.GlobalAveragePooling2D(),

#         #tf.keras.layers.Dense(128, activation='relu'),
#         #tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.Dense(5, activation='softmax')
# ])
    
# model.compile(
#         optimizer=tf.keras.optimizers.Adam(),
#         loss='sparse_categorical_crossentropy',  
#         metrics=['sparse_categorical_accuracy']
# )

#model.load_weights('../input/getting-started-tpus-new-tfrecords-01052020-v2/resnet50_01052021_v2.h5')
# model101_0 = keras.models.load_model('../input/tpus-resnet101-with-5-fold/resnet101_0.h5')# This one is no good
# model101_1 = keras.models.load_model('../input/tpus-resnet101-with-5-fold/resnet101_1.h5')
# model101_2 = keras.models.load_model('../input/tpus-resnet101-with-5-fold/resnet101_2.h5')
# model101_3 = keras.models.load_model('../input/tpus-resnet101-with-5-fold/resnet101_3.h5')
# model101_4 = keras.models.load_model('../input/tpus-resnet101-with-5-fold/resnet101_4.h5')

# model50_0 = keras.models.load_model('../input/resnet50-5fold-0/resnet50_0.h5')
# model50_1 = keras.models.load_model('../input/resnet50-5fold-0/resnet50_1.h5')
# model50_2 = keras.models.load_model('../input/resnet50-5fold-0/resnet50_2.h5')
# model50_3 = keras.models.load_model('../input/resnet50-5fold-0/resnet50_3.h5')
# model50_4 = keras.models.load_model('../input/resnet50-5fold-0/resnet50_4.h5')

# modeleffb3_0 = keras.models.load_model('../input/efficient-net-0115/efficientnet_0.h5')

# modeleffb4_0 = keras.models.load_model('../input/efficientb4-net-0117/efficientnet_0.h5')
modeleffb4_0121 = keras.models.load_model('../input/efficientb4-net-0121/efficientnet_0.h5')
modeleffb4_0124 = keras.models.load_model('../input/fork-of-fork-of-efficientb4-net-0124/efficientnet_0.h5')

mod_lst = [modeleffb4_0124]
# mod_lst = [model50_0, model50_1, model50_2, model50_3, model50_4]
# mod_lst = [model101_1, model101_2]
# mod_lst = [modeleffb4_0, model50_4, model101_4]

In [ ]:
test_dir = "../input/cassava-leaf-disease-classification/test_images"

# The image augmentations to use:

In [ ]:
seq = iaa.Sequential([
    iaa.Crop(px=(0, 128)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Flipud(0.5), # vertically flip 50% of all images
    #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
#     iaa.GaussianBlur(sigma=(0, 2.0)), # blur images with a sigma of 0 to 3.0
#     iaa.Dropout((0.01, 0.15), per_channel=0.5)
])

In [ ]:
def get_preds_model_list_norm_inds(image_dir, model_obj_list, TTA=True, aug_num=5, normalize_indices=None):
    '''
    normalize_indices should be a list of 0s and 1s for indicators of which models we want to have normalized (i.e., divided by 255)
    
    '''
    
    preds = []
    img_ids = []
    
    for i in glob.glob(image_dir+"/*.jpg"):
        print(i)
        image = Image.open(i)
        image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
        
        # create aug_num augmentations of the image and let all models in model_obj_list get a score
        aug_imgs = [seq(image=np.array(image))*((1-g)+g/255) for j in range(aug_num) for g in normalize_indices]
            
        avg_pred = np.concatenate([mod.predict(np.expand_dims(aug_imgs[i], axis = 0)) for i in range(aug_num) for mod in model_obj_list]).mean(0)

        
        preds.append(np.argmax(avg_pred))
        img_ids.append(i.replace(image_dir+"/", ""))
    
    return(pd.DataFrame({'image_id':img_ids, 
                         'label':preds}))

In [ ]:
def get_preds_model_list(image_dir, model_obj_list, TTA=True, aug_num=5, normalize=True):
    preds = []
    img_ids = []
    
    for i in glob.glob(image_dir+"/*.jpg"):
        print(i)
        image = Image.open(i)
        image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
        #image = np.expand_dims(image, axis = 0)
        #image = image/255
        
    
        if(TTA):
            # create aug_num augmentations of the image and let all models in model_obj_list get a score
            if normalize:
                aug_imgs = [seq(image=np.array(image))/255 for j in range(aug_num)]
            else: 
                aug_imgs = [seq(image=np.array(image)) for j in range(aug_num)]
            #aug_imgs = np.expand_dims(aug_imgs, axis = 0)
            avg_pred = np.concatenate([mod.predict(np.expand_dims(aug_imgs[i], axis = 0)) for i in range(aug_num) for mod in model_obj_list]).mean(0)
        else:
            image = image/255
            # take the average of all models' predictions
            avg_pred = np.concatenate([mod.predict(image) for mod in model_obj_list]).mean(0)
        
        preds.append(np.argmax(avg_pred))
        img_ids.append(i.replace(image_dir+"/", ""))
    
    return(pd.DataFrame({'image_id':img_ids, 
                         'label':preds}))

In [ ]:
def get_preds(image_dir, model_obj, normalize=True):
    preds = []
    img_ids = []
    
    for i in glob.glob(image_dir+"/*.jpg"):
        image = Image.open(i)
        image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
        image = np.expand_dims(image, axis = 0)
        
        if normalize:
            image = image/255
        preds.append(np.argmax(model_obj.predict(image)))
        img_ids.append(i.replace(image_dir+"/", ""))
    
    return(pd.DataFrame({'image_id':img_ids, 
                         'label':preds}))

In [ ]:


predict_df = get_preds_model_list(test_dir, mod_lst, normalize=False, aug_num=5)
# predict_df = get_preds(test_dir, model_obj=modeleffb4_0, normalize=False)
# predict_df = get_preds_model_list_norm_inds(test_dir, mod_lst, normalize_indices=[0,1,1])

predict_df.to_csv("submission.csv", index=False)

In [ ]:
display(predict_df)